# Bước 2: Làm sạch & Chuẩn hoá (Phiên bản đã sửa lỗi)

Phiên bản này sửa 3 vấn đề chính từ file gốc:
1.  **Lỗi logic gắn cờ (Flagging):** Sử dụng các cột cờ riêng biệt (ví dụ: `flag_missing_text`) thay vì một cột `flag` chung bị ghi đè.
2.  **Lỗi phân tách Vị trí (Location):** Dùng `.str.split()` để phân tách `lat` và `lon` chính xác, thay vì regex làm mất dấu âm.
3.  **Thiếu sót:** Bổ sung bước xử lý tệp `keywords.csv` (làm sạch, xoá trùng lặp) theo yêu cầu của đề bài.

### 0. Tải thư viện & Thiết lập

In [1]:
import pandas as pd
import numpy as np
import re
import os

print("Bắt đầu Bước 2 (Phiên bản đã sửa lỗi)...")

# --- 0. Thiết lập: Định nghĩa đường dẫn ---
# Giả sử bạn chạy file này từ thư mục 'src/' hoặc 'notebooks/'
# Đường dẫn sẽ là '../raw', '../processed', ...
try:
    # Nếu chạy trong notebook, os.getcwd() thường là thư mục chứa notebook
    BASE_DIR = os.getcwd()
    RAW_DIR = os.path.join(BASE_DIR, "..", "raw")
    PROCESSED_DIR = os.path.join(BASE_DIR, "..", "processed")
    REPORTS_DIR = os.path.join(BASE_DIR, "..", "reports")
    
    # Kiểm tra lại đường dẫn
    if not os.path.exists(RAW_DIR):
        print("Không tìm thấy thư mục 'raw'. Đang thử dùng đường dẫn tương đối...")
        RAW_DIR = "../raw"
        PROCESSED_DIR = "../processed"
        REPORTS_DIR = "../reports"
        
except Exception:
    RAW_DIR = "../raw"
    PROCESSED_DIR = "../processed"
    REPORTS_DIR = "../reports"

# Tạo thư mục nếu chúng chưa tồn tại
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

print(f"Thư mục RAW: {os.path.abspath(RAW_DIR)}")
print(f"Thư mục PROCESSED: {os.path.abspath(PROCESSED_DIR)}")
print(f"Thư mục REPORTS: {os.path.abspath(REPORTS_DIR)}")

# Định nghĩa khoảng thời gian cho Nhóm 2.4
START_DATE = "2011-05-12 00:00:00"
END_DATE = "2011-05-15 23:59:59"

# Danh sách (list) để lưu trữ các kết quả QA (Kiểm tra chất lượng)
qa_results = []

# Biến global để lưu số lượng hàng ban đầu
initial_rows_micro = 0
initial_rows_weather = 0
initial_keywords = 0

Bắt đầu Bước 2 (Phiên bản đã sửa lỗi)...
Thư mục RAW: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\raw
Thư mục PROCESSED: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\processed
Thư mục REPORTS: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\reports


### 1. Xử lý tệp microblogs.csv

In [2]:
print(f"\nĐang xử lý microblogs.csv (Lọc từ {START_DATE} đến {END_DATE})...")
microblog_path = os.path.join(RAW_DIR, "microblogs.csv")

try:
    df_micro = pd.read_csv(microblog_path, encoding='latin-1')

    # A. Chuẩn hoá Thời gian
    df_micro = df_micro.rename(columns={'Created_at': 'timestamp'})
    df_micro['timestamp'] = pd.to_datetime(df_micro['timestamp'], errors='coerce')

    # B. Lọc dữ liệu cho Nhóm 2.4 (12/5 - 15/5)
    micro_sub = df_micro[
        (df_micro['timestamp'] >= START_DATE) & (df_micro['timestamp'] <= END_DATE)
    ].copy()
    
    initial_rows_micro = len(micro_sub)
    print(f"Đã lọc được {initial_rows_micro} microblogs.")

    # C. Chuẩn hoá Văn bản
    micro_sub['text_clean'] = micro_sub['text'].fillna('').astype(str)
    micro_sub['text_clean'] = micro_sub['text_clean'].str.lower()
    # SỬA LỖI REGEX: Dùng \s+ thay vì \\s+
    micro_sub['text_clean'] = micro_sub['text_clean'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

    # D. SỬA LỖI: Chuẩn hoá Vị trí (Location)
    # Dùng .str.split() thay vì regex bị lỗi
    print("Đang chuẩn hoá vị trí (Location)...")
    location_split = micro_sub['Location'].str.split(r'\s+', expand=True)
    micro_sub['lat'] = pd.to_numeric(location_split[0], errors='coerce')
    micro_sub['lon'] = pd.to_numeric(location_split[1], errors='coerce')

    # E. SỬA LỖI LOGIC: Gắn cờ (Flagging)
    print("Đang gắn cờ dữ liệu lỗi...")
    # Dùng các cột cờ riêng biệt
    micro_sub['flag_bad_timestamp'] = micro_sub['timestamp'].isna()
    micro_sub['flag_missing_text'] = micro_sub['text_clean'] == ''
    
    # Cờ này đúng vì Location gốc là NaN
    micro_sub['flag_missing_location'] = micro_sub['Location'].isna() 
    
    # Cờ này bắt các trường hợp Location không NaN nhưng không thể parse
    micro_sub['flag_invalid_location'] = (micro_sub['lat'].isna() | micro_sub['lon'].isna()) & (~micro_sub['flag_missing_location'])

    # Ghi lại kết quả QA
    qa_results.append({'table': 'microblogs', 'flag': 'flag_bad_timestamp', 'count': micro_sub['flag_bad_timestamp'].sum()})
    qa_results.append({'table': 'microblogs', 'flag': 'flag_missing_text', 'count': micro_sub['flag_missing_text'].sum()})
    qa_results.append({'table': 'microblogs', 'flag': 'flag_missing_location', 'count': micro_sub['flag_missing_location'].sum()})
    qa_results.append({'table': 'microblogs', 'flag': 'flag_invalid_location', 'count': micro_sub['flag_invalid_location'].sum()})

    # F. Lưu kết quả
    micro_clean_path = os.path.join(PROCESSED_DIR, "microblogs_clean.csv")
    micro_sub.to_csv(micro_clean_path, index=False, encoding='utf-8')
    print(f"Đã lưu microblogs đã làm sạch vào: {micro_clean_path}")

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy tệp {microblog_path}")
except Exception as e:
    print(f"Đã xảy ra lỗi khi xử lý microblogs.csv: {e}")


Đang xử lý microblogs.csv (Lọc từ 2011-05-12 00:00:00 đến 2011-05-15 23:59:59)...
Đã lọc được 182422 microblogs.
Đang chuẩn hoá vị trí (Location)...
Đang gắn cờ dữ liệu lỗi...
Đã lưu microblogs đã làm sạch vào: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\src\..\processed\microblogs_clean.csv


### 2. Xử lý tệp Weather.csv

In [3]:
print(f"\nĐang xử lý Weather.csv (Lọc từ {START_DATE} đến {END_DATE})...")
weather_path = os.path.join(RAW_DIR, "Weather.csv")

try:
    df_weather = pd.read_csv(weather_path, encoding='latin-1')

    # A. Chuẩn hoá Thời gian
    df_weather = df_weather.rename(columns={'Date': 'timestamp'})
    df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'], errors='coerce')

    # B. Lọc dữ liệu cho Nhóm 2.4
    weather_sub = df_weather[
        (df_weather['timestamp'] >= START_DATE) & (df_weather['timestamp'] <= END_DATE)
    ].copy()
    initial_rows_weather = len(weather_sub)
    print(f"Đã lọc được {initial_rows_weather} bản ghi thời tiết.")

    # C. Chuẩn hoá Danh mục (Logic tốt từ file của bạn)
    weather_sub['Weather'] = weather_sub['Weather'].str.strip().str.upper()
    weather_sub['Wind_Direction'] = weather_sub['Wind_Direction'].str.strip().str.upper()

    weather_map = {
        'CLEAR': 'SUNNY', 'SUN': 'SUNNY', 'RAINY': 'RAIN', 'SHOWERS': 'RAIN',
        'OVERCAST': 'CLOUDY', 'STORMY': 'STORM', 'FOGGY': 'FOG'
    }
    weather_sub['Weather'] = weather_sub['Weather'].replace(weather_map)

    valid_weather = ['SUNNY', 'CLOUDY', 'RAIN', 'STORM', 'FOG']
    weather_sub.loc[~weather_sub['Weather'].isin(valid_weather), 'Weather'] = 'UNKNOWN'

    # D. SỬA LỖI LOGIC: Gắn cờ (Flagging)
    weather_sub['flag_bad_timestamp'] = weather_sub['timestamp'].isna()
    weather_sub['flag_missing_weather'] = weather_sub['Weather'].isin(['UNKNOWN', np.nan])
    weather_sub['flag_missing_wind_dir'] = weather_sub['Wind_Direction'].isna() | (weather_sub['Wind_Direction'].str.strip() == '')
    weather_sub['flag_negative_wind_speed'] = weather_sub['Average_Wind_Speed'] < 0
    weather_sub['flag_abnormal_wind_speed'] = weather_sub['Average_Wind_Speed'] > 200 # Ngưỡng tuỳ chỉnh

    # Ghi lại kết quả QA
    qa_results.append({'table': 'weather', 'flag': 'flag_bad_timestamp', 'count': weather_sub['flag_bad_timestamp'].sum()})
    qa_results.append({'table': 'weather', 'flag': 'flag_missing_weather', 'count': weather_sub['flag_missing_weather'].sum()})
    qa_results.append({'table': 'weather', 'flag': 'flag_missing_wind_dir', 'count': weather_sub['flag_missing_wind_dir'].sum()})
    qa_results.append({'table': 'weather', 'flag': 'flag_negative_wind_speed', 'count': weather_sub['flag_negative_wind_speed'].sum()})
    qa_results.append({'table': 'weather', 'flag': 'flag_abnormal_wind_speed', 'count': weather_sub['flag_abnormal_wind_speed'].sum()})

    # E. Lưu kết quả
    weather_clean_path = os.path.join(PROCESSED_DIR, "weather_clean.csv")
    weather_sub.to_csv(weather_clean_path, index=False, encoding='utf-8')
    print(f"Đã lưu weather đã làm sạch vào: {weather_clean_path}")

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy tệp {weather_path}")
except Exception as e:
    print(f"Đã xảy ra lỗi khi xử lý Weather.csv: {e}")


Đang xử lý Weather.csv (Lọc từ 2011-05-12 00:00:00 đến 2011-05-15 23:59:59)...
Đã lọc được 4 bản ghi thời tiết.
Đã lưu weather đã làm sạch vào: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\src\..\processed\weather_clean.csv


### 3. BỔ SUNG: Xử lý tệp keywords.csv

In [4]:
print("\nĐang xử lý keywords.csv...")
keywords_path = os.path.join(RAW_DIR, "keywords.csv")

try:
    # Tệp của bạn có cột index thừa và không có header
    df_keywords = pd.read_csv(keywords_path, header=None, names=['index_col', 'keyword'])
    df_keywords = df_keywords.drop(columns=['index_col'])
    initial_keywords = len(df_keywords)

    # A. Chuẩn hoá Văn bản
    df_keywords['keyword_clean'] = df_keywords['keyword'].str.lower().str.strip()
    
    # B. QA (Kiểm tra từ khoá trùng lặp)
    duplicates = df_keywords['keyword_clean'].duplicated().sum()
    qa_results.append({
        'table': 'keywords',
        'flag': 'duplicate_keywords',
        'count': duplicates
    })
    
    # Xoá các từ khoá trùng lặp
    df_keywords_cleaned = df_keywords.drop_duplicates(subset=['keyword_clean'])
    print(f"Đã tìm thấy và loại bỏ {duplicates} từ khoá trùng lặp.")

    # C. Lưu kết quả
    keywords_clean_path = os.path.join(PROCESSED_DIR, "keywords_cleaned.csv")
    # Chỉ lưu cột đã làm sạch, và đặt tên header là 'keyword'
    df_keywords_cleaned[['keyword_clean']].to_csv(keywords_clean_path, index=False, header=['keyword'])
    print(f"Đã lưu keywords đã làm sạch vào: {keywords_clean_path}")

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy tệp {keywords_path}")
except Exception as e:
    print(f"Đã xảy ra lỗi khi xử lý keywords.csv: {e}")


Đang xử lý keywords.csv...
Đã tìm thấy và loại bỏ 1 từ khoá trùng lặp.
Đã lưu keywords đã làm sạch vào: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\src\..\processed\keywords_cleaned.csv


### 4. Tạo Báo cáo QA (qa_summary.csv)

In [5]:
print("\nĐang tạo báo cáo tổng hợp QA...")

if qa_results:
    qa_summary = pd.DataFrame(qa_results)
    
    # Thêm cột tổng số hàng cho từng bảng để tính phần trăm
    total_rows = {
        'microblogs': initial_rows_micro,
        'weather': initial_rows_weather,
        'keywords': initial_keywords
    }
    
    try:
        qa_summary['total_rows'] = qa_summary['table'].map(total_rows)
        # Tính phần trăm
        qa_summary['percent'] = ((qa_summary['count'] / qa_summary['total_rows']) * 100).round(2)
        # Sắp xếp lại cột
        qa_summary = qa_summary[['table', 'flag', 'count', 'total_rows', 'percent']]
    except Exception as e:
         print(f"Lỗi khi tính toán QA: {e}")
         print("Không thể tính 'total_rows' hoặc 'percent' do lỗi nạp tệp trước đó.")
         pass

    # Lưu tệp báo cáo
    qa_path = os.path.join(REPORTS_DIR, "qa_summary.csv")
    qa_summary.to_csv(qa_path, index=False, encoding='utf-8')
    
    print(f"Đã lưu báo cáo tổng hợp QA tại: {qa_path}")
    print("\nNội dung báo cáo QA:")
    print(qa_summary)
else:
    print("Không có kết quả QA để lưu.")

print("\n--- Bước 2 (Sửa lỗi): Hoàn thành ---")


Đang tạo báo cáo tổng hợp QA...
Đã lưu báo cáo tổng hợp QA tại: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\src\..\reports\qa_summary.csv

Nội dung báo cáo QA:
        table                      flag  count  total_rows  percent
0  microblogs        flag_bad_timestamp      0      182422     0.00
1  microblogs         flag_missing_text     36      182422     0.02
2  microblogs     flag_missing_location      0      182422     0.00
3  microblogs     flag_invalid_location      0      182422     0.00
4     weather        flag_bad_timestamp      0           4     0.00
5     weather      flag_missing_weather      0           4     0.00
6     weather     flag_missing_wind_dir      0           4     0.00
7     weather  flag_negative_wind_speed      0           4     0.00
8     weather  flag_abnormal_wind_speed      0           4     0.00
9    keywords        duplicate_keywords      1        1302     0.08

--- Bước 2 (Sửa lỗi): Hoàn thành ---


### 5. (Tùy chọn) Hiển thị kết quả

In [6]:
print("Hiển thị 5 hàng đầu của microblogs đã xử lý:")
display(micro_sub.head())

print("\nHiển thị 5 hàng đầu của weather đã xử lý:")
display(weather_sub.head())

print("\nHiển thị 5 hàng đầu của keywords đã xử lý:")
try:
    display(df_keywords_cleaned.head())
except NameError:
    print("Không thể hiển thị df_keywords_cleaned (có thể do lỗi nạp tệp). Gõ 'df_keywords_cleaned.head()' ở cell sau để thử lại.")

Hiển thị 5 hàng đầu của microblogs đã xử lý:


,ID,timestamp,Location,text,text_clean,lat,lon,flag_bad_timestamp,flag_missing_text,flag_missing_location,flag_invalid_location
2,3,2011-05-14 09:22:00,42.22479 93.35922,Nike 'gana' el Mundial al patrocinador oficial...,nike 'gana' el mundial al patrocinador oficial...,42.22479,93.35922,False,False,False,False
5,3,2011-05-12 09:22:00,42.27167 93.42951,adidas vs puma en managment tv. Muy interesante !,adidas vs puma en managment tv. muy interesante !,42.27167,93.42951,False,False,False,False
7,3,2011-05-13 17:21:00,42.2838 93.47745,unsure if I will ever feel comfortable flying ...,unsure if i will ever feel comfortable flying ...,42.28380,93.47745,False,False,False,False
16,4,2011-05-13 07:22:00,42.16877 93.40755,Skinny Cinnamon Dolce Latte from Starbucks ma...,skinny cinnamon dolce latte from starbucks mak...,42.16877,93.40755,False,False,False,False
18,4,2011-05-12 07:22:00,42.21211 93.38398,9. Lemon Cake from starbucks,9. lemon cake from starbucks,42.21211,93.38398,False,False,False,False



Hiển thị 5 hàng đầu của weather đã xử lý:


,timestamp,Weather,Average_Wind_Speed,Wind_Direction,flag_bad_timestamp,flag_missing_weather,flag_missing_wind_dir,flag_negative_wind_speed,flag_abnormal_wind_speed
12,2011-05-12,CLOUDY,8,N,False,False,False,False,False
13,2011-05-13,RAIN,10,N,False,False,False,False,False
14,2011-05-14,CLOUDY,8,NW,False,False,False,False,False
15,2011-05-15,SUNNY,8,NW,False,False,False,False,False



Hiển thị 5 hàng đầu của keywords đã xử lý:


,keyword,keyword_clean
0,0,0
1,from,from
2,newtwitter,newtwitter
3,life,life
4,u,u
